In [2]:
from ctapipe.io import HDF5TableWriter
from ctapipe.core import Container, Field
import numpy as np

In [3]:
class VariousTypesContainer(Container):
    
    a_int = Field(int, 'int')
    a_float = Field(float, 'float')
    a_bool = Field(bool, 'bool')
    a_np_int = Field(np.int, 'np.int')
    a_np_float = Field(np.float, 'np.float')
    a_np_bool = Field(np.bool, 'np.bool')

In [ ]:
def create_stream(n_event):
    
    data = VariousTypesContainer()
    for i in range(n_event):
        
        data.a_int = int(i)
        data.a_float = float(i)
        data.a_bool = (i % 2) == 0
        data.a_np_int = np.int(i)
        data.a_np_float = np.float(i)
        data.a_np_bool = np.bool((i % 2) == 0)
        
        yield data

In [ ]:
for data in create_stream(2):
        
    for key, val in data.items():
        
        print('{}: {}, type : {}'.format(key, val, type(val)))

# Writing the Data

## This is not recommended

In [ ]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)

h5_table.close()

In that case the file is not garenteed to close properly for instance if one does a mistake in the for loop. Let's just add a stupid mistake and see what happens.

In [ ]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)
    0/0

h5_table.close()

Now the file did not close properly. So let's try to correct the mistake and execute the code again.

In [ ]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)

h5_table.close()

Ah it seems that the file did not close! Now I am stuck. Maybe I should restart the kernel? ahh no I don't want to loose everything. Can I just close it ?

In [ ]:
h5_table.close()

It worked!

## Better to use context management

In [ ]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
        
    for data in create_stream(10):
    
        h5_table.write('table', data)
print('Done')

Now let me do the same mistake

In [ ]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
    
    for data in create_stream(10):
        
        h5_table.write('table', data)
        0 / 0
print('Done')

Let's fix it again...

In [ ]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
        
    for data in create_stream(10):
    
        h5_table.write('table', data)
        
print('Done')

Ok perfect my file closed automatically in this case!

In [ ]:
!ls container.h5

# Appending new Containers

To append some new containers we need to set the writing in append mode by using: 'mode=a'. But let's now first look at what happens if we don't.

In [ ]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='w', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

In [ ]:
!rm container.h5

Ok so the writer destroyed the content of the file each time it opens the file. Now let's try to append some data group to it! (using mode='a')

In [ ]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='a', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

So we can append some data groups. As long as the data group_name does not already exists. Let's try to overwrite the data group : data_1

In [ ]:
with HDF5TableWriter('container.h5', mode='a', group_name='data_1') as h5_table:
    
    for data in create_stream(10):
    
        h5_table.write('table', data)

Good ! I cannot overwrite my data.

In [ ]:
print(bool(h5_table._h5file.isopen))

# Reading the Data

In [7]:
from ctapipe.io import HDF5TableReader

def read(mode):
    
    print('reading mode {}'.format(mode))

    with HDF5TableReader('container.h5', mode=mode) as h5_table:

        for group_name in ['data_0/', 'data_1/']:

            group_name = '/{}table'.format(group_name)
            print(group_name)

            for data in h5_table.read(group_name, VariousTypesContainer()):

                print(data.as_dict())
                
read('r')
read('r+')
read('a')
read('w')

reading mode r
/data_0/table
{'a_bool': True, 'a_np_bool': True, 'a_float': 0.0, 'a_np_float': 0.0, 'a_np_int': 0, 'a_int': 0}
{'a_bool': False, 'a_np_bool': False, 'a_float': 1.0, 'a_np_float': 1.0, 'a_np_int': 1, 'a_int': 1}
{'a_bool': True, 'a_np_bool': True, 'a_float': 2.0, 'a_np_float': 2.0, 'a_np_int': 2, 'a_int': 2}
{'a_bool': False, 'a_np_bool': False, 'a_float': 3.0, 'a_np_float': 3.0, 'a_np_int': 3, 'a_int': 3}
{'a_bool': True, 'a_np_bool': True, 'a_float': 4.0, 'a_np_float': 4.0, 'a_np_int': 4, 'a_int': 4}
{'a_bool': False, 'a_np_bool': False, 'a_float': 5.0, 'a_np_float': 5.0, 'a_np_int': 5, 'a_int': 5}
{'a_bool': True, 'a_np_bool': True, 'a_float': 6.0, 'a_np_float': 6.0, 'a_np_int': 6, 'a_int': 6}
{'a_bool': False, 'a_np_bool': False, 'a_float': 7.0, 'a_np_float': 7.0, 'a_np_int': 7, 'a_int': 7}
{'a_bool': True, 'a_np_bool': True, 'a_float': 8.0, 'a_np_float': 8.0, 'a_np_int': 8, 'a_int': 8}
{'a_bool': False, 'a_np_bool': False, 'a_float': 9.0, 'a_np_float': 9.0, 'a_np_in

OSError: File container.h5 cannot be read in write mode "w"